# Text-to-SQL Agent Tutorial

This notebook demonstrates how to use the text-to-SQL agent built with LangChain's `create_agent`.

The agent can answer natural language questions about the Chinook database (a digital media store).

## Setup

Make sure you have:
1. Installed dependencies: `uv pip install -e`
2. Created a `.env` file with your `ZHIPU_API_KEY` and `ZHIPU_BASE_URL`
3. (Optional) Added LangSmith credentials for tracing

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# Load environment variables
load_dotenv()

In [ ]:
# System prompt for the SQL agent
SYSTEM_PROMPT = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
"""

In [ ]:
# Connect to Chinook database
db_path = os.path.join(os.getcwd(), "chinook.db")
db = SQLDatabase.from_uri(
    f"sqlite:///{db_path}",
    sample_rows_in_table_info=3
)

print(f"Connected to database: {db_path}")
print(f"Database dialect: {db.dialect}")

In [ ]:
# Initialize GLM-5 (Zhipu AI)
model = ChatOpenAI(
    model="glm-5",
    openai_api_key=os.getenv("ZHIPU_API_KEY"),
    openai_api_base=os.getenv("ZHIPU_BASE_URL", "https://open.bigmodel.cn/api/paas/v4/"),
    temperature=0.3,
)

print("Model initialized: glm-5")

In [ ]:
# Create SQL toolkit with tools
toolkit = SQLDatabaseToolkit(db=db, llm=model)
tools = toolkit.get_tools()

print(f"Available tools: {[tool.name for tool in tools]}")

In [ ]:
# Create the agent
agent = create_agent(
    model,
    tools,
    system_prompt=SYSTEM_PROMPT.format(dialect=db.dialect, top_k=5)
)

print("Agent created successfully!")

## Helper Function

Let's create a helper function to run queries and display results nicely.

In [ ]:
def ask(question: str):
    """Ask the agent a question about the database"""
    print(f"\n{'='*80}")
    print(f"Question: {question}")
    print('='*80)
    
    result = agent.invoke({
        "messages": [{"role": "user", "content": question}]
    })
    
    answer = result["messages"][-1].content
    print(f"\nAnswer:\n{answer}\n")
    
    return result

## Example 1: Simple Count Query

Let's start with a simple query to count customers from a specific country.

In [ ]:
ask("How many customers are from Canada?")

## Example 2: Aggregation with GROUP BY

Now let's try a more complex query with aggregation.

In [ ]:
ask("What is the total revenue by country?")

## Example 3: Complex Query with Multiple JOINs

Let's try something more complex that requires joining multiple tables.

In [ ]:
ask("What are the top 5 best-selling tracks?")

## LangSmith Tracing

If you have LangSmith configured in your `.env` file, every query is automatically traced and you can see detailed execution logs.

### Check LangSmith Configuration

Run the cell below to verify your LangSmith setup:

In [ ]:
# Check LangSmith configuration
print("LangSmith Configuration:")
print(f"  Tracing Enabled: {os.getenv('LANGCHAIN_TRACING_V2', 'false')}")
print(f"  API Key: {'[Set]' if os.getenv('LANGCHAIN_API_KEY') else '[Not Set]'}")
print(f"  Project: {os.getenv('LANGCHAIN_PROJECT', 'default')}")
print(f"  Endpoint: {os.getenv('LANGSMITH_ENDPOINT', 'default (https://api.smith.langchain.com)')}")

if os.getenv('LANGCHAIN_TRACING_V2') == 'true' and os.getenv('LANGCHAIN_API_KEY'):
    print("\n[OK] LangSmith is configured! Traces will be sent to: https://smith.langchain.com/")
else:
    print("\n[WARNING] LangSmith is not fully configured. Add these to your .env file:")
    print("   LANGCHAIN_TRACING_V2=true")
    print("   LANGCHAIN_API_KEY=your_key_here")
    print("   LANGCHAIN_PROJECT=text2sql-agent")

In [ ]:
#run the same question again but see the trace in LangSmith
ask("What are the top 5 best-selling tracks?")

#### Now let's check out the traces in LangSmith to see what's going on

In [ ]:
from IPython.display import Image, display
  
display(Image('text2sql-LangSmithTraceView.png'))

## Database Schema Reference

The Chinook database contains these tables:

- **Album** - Music albums
- **Artist** - Music artists
- **Customer** - Store customers
- **Employee** - Store employees
- **Genre** - Music genres
- **Invoice** - Customer invoices
- **InvoiceLine** - Individual items on invoices
- **MediaType** - Types of media (MP3, AAC, etc.)
- **Playlist** - Music playlists
- **PlaylistTrack** - Tracks in playlists
- **Track** - Individual music tracks